In [82]:
import numpy as np
from sklearn.cluster import  KMeans
from sklearn.manifold import TSNE
import torch
import argparse
import json
from tqdm import tqdm
import pandas as pd
from FlagEmbedding import FlagModel
import time
import submodlib
from submodlib.functions.facilityLocation import FacilityLocationFunction
from datasets import Dataset

In [83]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--pt_data_path", type=str, required=True)
    parser.add_argument("--json_data_path", type=str, required=True)
    parser.add_argument("--json_save_path", type=str, required=True)
    parser.add_argument("--sent_type", type=int, default=0)
    parser.add_argument("--ppl_type", type=int, default=0)
    parser.add_argument("--cluster_method", type=str, default='kmeans')
    parser.add_argument("--reduce_method", type=str, default='tsne')
    parser.add_argument("--sample_num", type=int, default=10)
    parser.add_argument("--kmeans_num_clusters", type=int, default=100)
    parser.add_argument("--low_th", type=int, default=1)
    parser.add_argument("--up_th", type=int, default=99)

    args = parser.parse_args()
    return args

def do_clustering(high_dim_vectors,cluster_method='kmeans',kmeans_num_clusters=100):

    clustering_algorithm = cluster_method
    if clustering_algorithm == 'kmeans':
        clustering = KMeans(n_clusters=kmeans_num_clusters, random_state=0).fit(high_dim_vectors)
    
    return clustering

def do_reduce_dim(high_dim_vectors):
    # Perform t-SNE for visualization
    # if args.reduce_method == 'tsne':
    tsne = TSNE(n_components=2, random_state=0)
    low_dim_vectors = tsne.fit_transform(high_dim_vectors)
    return low_dim_vectors

def sample_middle_confidence_data(cluster_labels, confidences, n, low_th=25, up_th=75):
    num_clusters = len(np.unique(cluster_labels))

    # Get the indices for each cluster
    cluster_indices = {i: np.where(cluster_labels == i)[0] for i in range(num_clusters)}
    
    # Create a dictionary to store the indices of the middle level confidence samples
    middle_confidence_samples = {}

    for i in range(num_clusters):
        # Get the sorted indices for this cluster
        sorted_indices = cluster_indices[i]
        
        # If there are less than n samples in this class, just return all of them
        if len(sorted_indices) < n:
            middle_confidence_samples[i] = sorted_indices
            continue

        # Get the confidences for this cluster
        cluster_confidences = confidences[sorted_indices]
        lower_threshold = np.percentile(cluster_confidences, low_th)
        upper_threshold = np.percentile(cluster_confidences, up_th)

        # Get the indices of the samples within the middle level confidence range
        middle_indices = sorted_indices[(cluster_confidences >= lower_threshold) & (cluster_confidences <= upper_threshold)]
        
        # If there are less than n samples in the middle range, use all of them
        if len(middle_indices) < n:
            middle_confidence_samples[i] = middle_indices
        else:
            # Calculate step size for even sampling
            step_size = len(middle_indices) // n
            # Select evenly from the middle level confidence samples
            middle_confidence_samples[i] = middle_indices[::step_size][:n]

    return middle_confidence_samples

### Get Embeddings by SentBert Model

### Parse LLM File

In [84]:
All_Data = pd.read_json('/data/home/wangys/transfer-er/Pipeline/Amazon-Google/LLM_file/Amazon-Google-Train-Match-P1.json')

In [9]:
def cut_input_length(row):
    input = row['instruction']
    output = input.split('at the final judgement.')[1].split('Take these examples as reference:')[0]
    return output
All_Data['instruction'] = All_Data.apply(cut_input_length,axis=1)

In [85]:
# text = All_Data.iloc[0,0]
left_list = []
right_list = []
label_list = []
for index,row in All_Data.iterrows():
    text = row[0]
    Entity_1 = text.split('\n\nEntity 1:')[1].split('\n\nEntity 2')[0]
    Entity_2 = text.split('\n\nEntity 2:')[1].split('\n\nTake these examples as reference:')[0] 
    label = eval(row[-1])['Output']
    left_list.append(Entity_1)
    right_list.append(Entity_2)
    label_list.append(label)

/tmp/ipykernel_2106371/2444698609.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = row[0]
/tmp/ipykernel_2106371/2444698609.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = eval(row[-1])['Output']


In [86]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='6'
model = FlagModel('../sentence_transformer_model/bge-large-en-1.5/', 
                  use_fp16=True)
embedding_a = model.encode(left_list)
embedding_b = model.encode(right_list)
embedding_c = model.encode(label_list)

Inference Embeddings: 100%|██████████| 15/15 [00:00<00:00, 79.08it/s]


In [87]:
# ppl = pd.read_json('/data/home/wangys/DataSelection-IF/ppl/ppl_qwen2.5-7B-AG-Short.json')
ppl = pd.read_csv('ppl/ppl_qwen2.5-0.5B-AG-short.csv',index_col=0)

In [88]:
import numpy as np
ppl_list = ppl.iloc[:,0].to_list()
pt_data = np.concatenate([embedding_a,embedding_b,embedding_c],axis=1)
pt_data.shape

(3663, 3072)

In [89]:
# args = parse_args()
# print(args)

# pt_data = torch.load(args.pt_data_path, map_location=torch.device('cpu'))
# with open(args.json_data_path, "r") as f:
#     json_data = json.load(f)
file_path = '/data/home/wangys/transfer-er/Pipeline/Amazon-Google/LLM_file/Amazon-Google-Train-Match-P1-wo-RAG.json'
json_data = pd.read_json(file_path)

# emb_list = []
# ppl_list = []
# for i in tqdm(range(len(json_data))):
#     sent_emb_list = pt_data[i]
#     # sent_emb_list = data_i['sent_emb']
#     emb_list.append(sent_emb_list)
#     ppl_list.append(ppl_list[i])
high_dim_vectors = pt_data

# high_dim_vectors = torch.cat(emb_list,0).numpy()
ppl_array = np.array(ppl_list)

clustering = do_clustering(high_dim_vectors,kmeans_num_clusters=100)
cluster_labels = clustering.labels_

def get_json_sample(middle_confidence_samples):
    
    json_samples = []
    for k in middle_confidence_samples.keys():
        ids_list = middle_confidence_samples[k].tolist()
        # for id_i in ids_list:
            # ori_sample = json_data[id_i]
        json_samples.extend(ids_list)
    

    return json_samples

# middle_confidence_samples = sample_middle_confidence_data(cluster_labels, ppl_array, n = 10, low_th=25, up_th = 75)
middle_confidence_samples = sample_middle_confidence_data(cluster_labels, ppl_array, n = 10, low_th=25, up_th = 75)

new_data = get_json_sample(middle_confidence_samples)
json_data.iloc[new_data]['output'].value_counts()
# print('New data len \n',len(new_data))
# with open(args.json_save_path, "w") as fw:
#     json.dump(new_data, fw, indent=4)
# pass


output
{'Output': 'mismatch'}    741
{'Output': 'match'}       250
Name: count, dtype: int64

In [53]:
json.dump(json_data.iloc[new_data].to_dict(orient='records'), open('train/AG-train-init.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### Calculate FL score

In [91]:
cluster_indices = {i: np.where(cluster_labels == i)[0] for i in range(100)}
# cluster_indices[0]

In [101]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering


# def cluster_vectors(high_dim_vectors_cluster, indexes, batch_size):
#     """
#     根据给定的高维向量矩阵、索引以及期望的每个聚类元素个数进行聚类划分。

#     参数:
#     high_dim_vectors_cluster (numpy.ndarray): m * n的高维向量矩阵，m为元素个数，n为向量维度。
#     indexes (numpy.ndarray): 对应高维向量矩阵中元素的索引列表，长度为k。
#     batch_size (int): 期望每个聚类包含的元素个数。

#     返回:
#     dict: 聚类划分后的结果，键为聚类编号（从0到k - 1），值为对应聚类包含的元素索引列表。
#     """
#     # 获取对应索引的向量
#     selected_vectors = high_dim_vectors_cluster[indexes]

#     # 计算这些向量之间的余弦相似度矩阵
#     similarity_matrix = cosine_similarity(selected_vectors)

#     # 使用层次聚类（AgglomerativeClustering）基于余弦相似度进行聚类划分
#     k = len(indexes)
#     clustering_model = AgglomerativeClustering(n_clusters=k, metric='cosine', linkage='average')
#     clustering_model.fit(similarity_matrix)

#     # 根据聚类结果分配所有的m个元素到对应的聚类中
#     cluster_assignments = {i: [] for i in range(k)}
#     for i in range(len(high_dim_vectors_cluster)):
#         vector = high_dim_vectors_cluster[i].reshape(1, -1)
#         similarities = cosine_similarity(vector, selected_vectors)[0]
#         closest_cluster = np.argmax(similarities)
#         cluster_assignments[closest_cluster].append(i)

#     # 调整每个聚类中的元素个数尽量接近batch_size
#     for cluster_id in range(k):
#         cluster_indices = cluster_assignments[cluster_id]
#         if len(cluster_indices) > batch_size:
#             # 如果当前聚类元素个数大于batch_size，可考虑进一步划分等策略（这里简单取前batch_size个元素示例）
#             cluster_assignments[cluster_id] = cluster_indices[:batch_size]
#         elif len(cluster_indices) < batch_size:
#             # 如果小于batch_size，可以从其他聚类补充等（这里暂不实现复杂补充逻辑，仅打印提示）
#             print(f"Cluster {cluster_id} has less than {batch_size} elements.")

#     return cluster_assignments
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering


def cluster_vectors(high_dim_vectors_cluster, indexes, batch_size):
    """
    根据给定的高维向量矩阵、索引以及期望的每个聚类元素个数进行聚类划分。

    参数:
    high_dim_vectors_cluster (numpy.ndarray): m * n的高维向量矩阵，m为元素个数，n为向量维度。
    indexes (numpy.ndarray): 对应高维向量矩阵中元素的索引列表，长度为k。
    batch_size (int): 期望每个聚类包含的元素个数。

    返回:
    tuple: 包含两个元素，第一个元素是聚类划分后的结果（字典形式，键为聚类编号，值为对应聚类包含的元素索引列表），
           第二个元素是覆盖率（float类型，表示已分配元素占总元素的比例）。
    """
    # 获取对应索引的向量
    selected_vectors = high_dim_vectors_cluster[indexes]

    # 计算这些向量之间的余弦相似度矩阵
    similarity_matrix = cosine_similarity(selected_vectors)

    # 使用层次聚类（AgglomerativeClustering）基于余弦相似度进行聚类划分
    k = len(indexes)
    clustering_model = AgglomerativeClustering(n_clusters=k, metric='cosine', linkage='average')
    clustering_model.fit(similarity_matrix)

    # 根据聚类结果分配所有的m个元素到对应的聚类中
    cluster_assignments = {i: [] for i in range(k)}
    for i in range(len(high_dim_vectors_cluster)):
        vector = high_dim_vectors_cluster[i].reshape(1, -1)
        similarities = cosine_similarity(vector, selected_vectors)[0]
        closest_cluster = np.argmax(similarities)
        cluster_assignments[closest_cluster].append(i)

    # 检查未分配的元素，并根据距离的就近原则分配到对应的聚类中
    all_indices = set(range(len(high_dim_vectors_cluster)))
    assigned_indices = set([index for sublist in cluster_assignments.values() for index in sublist])
    unassigned_indices = all_indices - assigned_indices
    for index in unassigned_indices:
        vector = high_dim_vectors_cluster[index].reshape(1, -1)
        distances = []
        for cluster_id in range(k):
            cluster_vectors = np.array([high_dim_vectors_cluster[i].reshape(1, -1) for i in cluster_assignments[cluster_id]])
            mean_cluster_vector = np.mean(cluster_vectors, axis=0)
            distance = cosine_similarity(vector, mean_cluster_vector)[0][0]
            distances.append(distance)
        closest_cluster = np.argmin(distances)
        cluster_assignments[closest_cluster].append(index)

    # 调整每个聚类中的元素个数尽量接近batch_size（这里简单处理，可根据实际优化）
    for cluster_id in range(k):
        cluster_indices = cluster_assignments[cluster_id]
        if len(cluster_indices) > batch_size:
            cluster_assignments[cluster_id] = cluster_indices[:batch_size]
        elif len(cluster_indices) < batch_size:
            while len(cluster_indices) < batch_size and unassigned_indices:
                # 从未分配元素中找距离当前聚类最近的补充进来
                index_to_add = None
                min_distance = float('inf')
                for unassigned_index in unassigned_indices:
                    unassigned_vector = high_dim_vectors_cluster[unassigned_index].reshape(1, -1)
                    mean_cluster_vector = np.mean([high_dim_vectors_cluster[i].reshape(1, -1) for i in cluster_indices], axis=0)
                    distance = cosine_similarity(unassigned_vector, mean_cluster_vector)[0][0]
                    if distance < min_distance:
                        min_distance = distance
                        index_to_add = unassigned_index
                if index_to_add is not None:
                    cluster_assignments[cluster_id].append(index_to_add)
                    unassigned_indices.remove(index_to_add)
                    cluster_indices = cluster_assignments[cluster_id]

    # 计算覆盖率
    coverage = len(assigned_indices) / len(high_dim_vectors_cluster)

    return cluster_assignments, coverage
def cosine_similarity_clustering(high_dim_vectors_cluster, indices, k, batch_size):
    """
    Perform k clustering based on cosine similarity, ensuring each cluster has batch_size elements.

    Parameters:
    - high_dim_vectors_cluster (np.ndarray): An m x n matrix containing m high-dimensional vectors.
    - indices (list): A list of k indices corresponding to m rows in the matrix.
    - k (int): Number of clusters.
    - batch_size (int): Desired number of elements per cluster.

    Returns:
    - clusters (list of lists): A list where each sublist contains the indices of the elements in a cluster.
    """
    m, n = high_dim_vectors_cluster.shape
    if len(indices) != k:
        raise ValueError("Number of provided indices must match the number of clusters (k).")

    # Step 1: Initialize cluster centers using the given indices
    cluster_centers = high_dim_vectors_cluster[indices]

    # Step 2: Compute cosine similarity between all elements and cluster centers
    similarity_matrix = cosine_similarity(high_dim_vectors_cluster, cluster_centers)

    # Step 3: Assign elements to clusters greedily (allow duplicates if needed)
    clusters = [[] for _ in range(k)]

    for _ in range(batch_size):  # Ensure each cluster has batch_size elements
        for cluster_idx in range(k):
            # Find the most similar element for the current cluster
            best_idx = -1
            best_similarity = -1
            for i in range(m):
                if similarity_matrix[i, cluster_idx] > best_similarity:
                    best_idx = i
                    best_similarity = similarity_matrix[i, cluster_idx]

            if best_idx != -1:
                clusters[cluster_idx].append(best_idx)

    # Step 4: Handle remaining elements by assigning to closest clusters
    remaining_elements = [i for i in range(m)]
    for i in remaining_elements:
        # Assign to the cluster with the highest similarity
        best_cluster = np.argmax(similarity_matrix[i])
        clusters[best_cluster].append(i)

    return clusters

In [93]:
def do_fla(X, number_all, number_select):
    start_time = time.time()

    Y = X
    obj = FacilityLocationFunction(n=number_all, mode="dense", data=Y, metric="cosine")
    greedyList = obj.maximize(budget=number_select, optimizer='LazyGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
    idx_list = [tuple_i[0] for tuple_i in greedyList]

    print('FLA time used:',(time.time()-start_time),'(second)')
    return idx_list,greedyList

# idx_list,greedyList = do_fla(high_dim_vectors,high_dim_vectors.shape[0],number_select=1000)

### Calculating FLA score per cluster

In [105]:
batch_size = 8
batch_division = {}
for i in range(100):
    batch_division[i] = []
    cluster_indice_index = cluster_indices[i]
    high_dim_vectors_cluster = high_dim_vectors[cluster_indice_index]
    cluster_size = len(cluster_indice_index)
    # print(i,cluster_size)
    fla_num = int(np.ceil(cluster_size / batch_size))
    idx_list,greedyList = do_fla(high_dim_vectors_cluster,high_dim_vectors_cluster.shape[0],number_select=fla_num) ## idx_list is the selected number
    result,coverage = cluster_vectors(high_dim_vectors_cluster,idx_list,batch_size)
    for cluster_ind in result.keys():
        global_result = [cluster_indice_index[j] for j in result[cluster_ind]] ## 将相对index映射到global index
        batch_division[i].append(global_result)
    # print(result,coverage)
    # result = cosine_similarity_clustering(high_dim_vectors_cluster,idx_list,k = fla_num, batch_size=batch_size)
    

FLA time used: 0.03293466567993164 (second)
FLA time used: 0.06013751029968262 (second)
FLA time used: 0.04508638381958008 (second)


[||||||||||||||||||||]100% [Iteration 13 of 13]

FLA time used: 0.025324106216430664 (second)
FLA time used: 0.02434849739074707 (second)
FLA time used: 0.006682872772216797 (second)
FLA time used: 0.011463642120361328 (second)
FLA time used: 0.0018830299377441406 (second)
FLA time used: 0.010698556900024414 (second)
FLA time used: 0.004029273986816406 (second)
FLA time used: 0.015811920166015625 (second)


[||||||||||||||||||||]100% [Iteration 7 of 7]

FLA time used: 0.003081798553466797 (second)
FLA time used: 0.007641315460205078 (second)
FLA time used: 0.001377105712890625 (second)
FLA time used: 0.0057103633880615234 (second)
FLA time used: 0.016421794891357422 (second)
FLA time used: 0.004853963851928711 (second)
FLA time used: 0.003812074661254883 (second)
FLA time used: 0.005659580230712891 (second)
FLA time used: 0.011415719985961914 (second)
FLA time used: 0.03092813491821289 (second)


...

[||||||||||||||||||||]100% [Iteration 7 of 7]

FLA time used: 0.0018758773803710938 (second)
FLA time used: 0.0035369396209716797 (second)
FLA time used: 0.0042836666107177734 (second)
FLA time used: 0.005993843078613281 (second)
FLA time used: 0.007248640060424805 (second)
FLA time used: 0.004332065582275391 (second)
FLA time used: 0.0024979114532470703 (second)
FLA time used: 0.003124237060546875 (second)
FLA time used: 0.033794403076171875 (second)
FLA time used: 0.007601499557495117 (second)
FLA time used: 0.013195991516113281 (second)


[||||||||||||||||||||]100% [Iteration 7 of 7]1]

FLA time used: 0.009653806686401367 (second)
FLA time used: 0.025946855545043945 (second)


[||||||||||||||||||||]100% [Iteration 10 of 10]

In [107]:
batch_sampler = []
for key in batch_division.keys():
    batch_sampler.extend(batch_division[key])

In [110]:
torch.save(batch_sampler,'ppl/AG-batch.pkl')

### 获取phase 2 的Cluster和Grad，计算IF分数，这里考虑使用一个python独立程序来计算
## 计算IF Score需要划分eval_dataset，这里需要修改cal_IF_self_divide.py来计算？